In [1]:
import os 

import fiona #must be import before geopandas
import geopandas as gpd
import rasterio
import rioxarray
import re
import rtree
import shapely
import pickle

#from cartopy import crs
import collections
import cv2
import math
from glob import glob
from tqdm.notebook import tqdm_notebook
import tqdm
# Standard packages
import tempfile
import warnings
import urllib
import shutil

# Less standard, but still pip- or conda-installable
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

from lxml.etree import Element,SubElement,tostring
import xml.dom.minidom
from xml.dom.minidom import parseString
import xml.etree.ElementTree as et
from xml.dom import minidom

#import requests
from PIL import Image
from io import BytesIO
import tqdm
from skimage.metrics import structural_similarity as compare_ssim
import imutils

import psutil

import data_eng.az_proc as ap
import data_eng.form_calcs as fc


#Parsing/Modifying XML
from lxml.etree import Element,SubElement,tostring
import xml.dom.minidom
from xml.dom.minidom import parseString
import xml.etree.ElementTree as et
from xml.dom import minidom

## File Paths

In [2]:
parent_directory = "//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//"

tile_names_tile_urls_complete_array = np.load("image_download_azure/tile_name_tile_url_complete_array.npy")

tiles_labeled = "tile_name_tile_url_labeled.npy"
tiles_labeled_from_complete_set = np.load(tiles_labeled)

tracker_file_path = 'outputs/tile_img_annotation_annotator.npy'
tile_img_annotation = np.load(tracker_file_path)
tile_img_annotation_annotator = np.load("outputs/tile_img_annotation_annotator.npy")

tiles_errors = 'tile_name_tile_url_error_downloading.npy'
tiles_errors = np.load(tiles_errors)

#create folder to hold tiles in completed dataset
#load tiles and tile xmls
tiles_complete_dataset_path = os.path.join(parent_directory,"complete_dataset","tiles")
os.makedirs(tiles_complete_dataset_path, exist_ok=True)
tiles_dir = tiles_complete_dataset_path

tiles_xml_complete_dataset_path = os.path.join(parent_directory,"complete_dataset","tiles_xml")
os.makedirs(tiles_xml_complete_dataset_path, exist_ok=True)
tiles_xml_dir = tiles_xml_complete_dataset_path
tiles_xml_list = os.listdir(tiles_xml_complete_dataset_path)


tile_names_tile_urls_complete_array_unique_standard_tile_names = np.load("tile_names_tile_urls_complete_array_unique_standard_tile_names.npy")

#NAIP quad map
quad_indicies_path = "C:/Users/rapiduser/Box/EPA STAR 2019 (Community Resistance to Environmental Disasters)/Data/AST Datasets/MapIndices_National_GDB/MapIndices_National_GDB.gdb"
#fiona.listlayers(quad_indicies_path)
rechipped_image_path = os.path.join(parent_directory,"temp/rechip")

folders_of_images_xmls_by_tile = os.listdir(rechipped_image_path)
images_and_xmls_by_tile_path = rechipped_image_path

# Add chips to rechip folder

In [3]:
#for tile_name in tqdm.tqdm(folders_of_images_xmls_by_tile):
#    add_chips_to_chip_folders(rechipped_image_path, tile_name)

# Reclassify narrow closed roof tanks and closed roof tanks by size

In [4]:
for tile in tqdm.tqdm(os.listdir(rechipped_image_path)):
    tile_annotation_path = os.path.join(rechipped_image_path, tile, "chips_positive_xml")
    for xml in os.listdir(tile_annotation_path):
        xml_path = os.path.join(tile_annotation_path, xml)
        fc.reclassify_narrow_closed_roof_and_closed_roof_tanks(xml_path)

100%|██████████| 2139/2139 [12:32<00:00,  2.84it/s]


# Generate table of characteristics for tiles/images

In [5]:
#tile_characteristics, image_characteristics = fc.image_tile_characteristics(rechipped_image_path, tiles_complete_dataset_path)
#tile_characteristics.to_csv("tile_characteristics.csv")
#image_characteristics.to_csv("image_characteristics.csv")
image_characteristics = pd.read_csv("image_characteristics.csv")
tile_characteristics = pd.read_csv("tile_characteristics.csv")

# Fix inconsistent labels 

In [6]:
#make consistent labels 
for tile_name in tqdm.tqdm(folders_of_images_xmls_by_tile):
    #specify image/xml paths for each tile
    positive_xml_dir = os.path.join(images_and_xmls_by_tile_path, tile_name, "chips_positive_xml")
    fc.correct_inconsistent_labels_xml(positive_xml_dir)

100%|██████████| 2139/2139 [07:07<00:00,  5.00it/s]


# Generate tile XMLs

In [ ]:
fc.generate_tile_xmls(rechipped_image_path, tiles_complete_dataset_path, tiles_xml_complete_dataset_path, int(512))

 50%|████▉     | 1063/2139 [30:40<11:50,  1.51it/s]  

## Combine neighboring bounding boxes

https://answers.opencv.org/question/231263/merging-nearby-rectanglesedited/
https://stackoverflow.com/questions/55593506/merge-the-bounding-boxes-near-by-into-one

In [ ]:
distance_limit = 5
tile_database = fc.merge_tile_annotations(tile_characteristics, tiles_xml_dir, distance_limit = 5)
#merged_tile_database =  tile_database[tile_database["merged_bbox"] == True]

# Clip storm surge data and append to tile level annotation database

In [ ]:
sc_slosh_inundation_map_path = "C:/Users/rapiduser/Box/EPA STAR 2019 (Community Resistance to Environmental Disasters)/Data/Natural Hazards Data/NHC Potential Storm Surge Flood Map/SC_SLOSH_MOM_Inundation"
tile_database = fc.identify_inundation_for_tanks(tile_database, sc_slosh_inundation_map_path)

In [ ]:
output_filepath = os.path.join(parent_directory,"complete_dataset","tile_level_annotations")
output_filename = 'tile_level_annotations'
fc.write_gdf(tile_database, output_filepath, output_filename)